In [31]:
import tensorflow as tf
import numpy as np
import shutil
import pandas as pd

from sklearn.preprocessing import LabelEncoder

In [40]:
train = pd.read_csv("/Users/jk/git/Study/Data/Dacon_dataset/movie_attendance/movies_train.csv")
train.head()

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num
0,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,NaN,0,91,2,23398
1,내부자들,(주)쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,1161602.50,2,387,3,7072501
2,은밀하게 위대하게,(주)쇼박스,액션,2013-06-05,123,15세 관람가,장철수,220775.25,4,343,4,6959083
3,나는 공무원이다,(주)NEW,코미디,2012-07-12,101,전체 관람가,구자홍,23894.00,2,20,6,217866
4,불량남녀,쇼박스(주)미디어플렉스,코미디,2010-11-04,108,15세 관람가,신근호,1.00,1,251,2,483387


In [41]:
categori_col = ["genre", "screening_rat"]

for col in categori_col:
    encoder = LabelEncoder()
    # train
    train[col] = encoder.fit_transform(train[col])

train = train.fillna(0)
train.head()

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num
0,개들의 전쟁,롯데엔터테인먼트,10,2012-11-22,96,3,조병옥,0.00,0,91,2,23398
1,내부자들,(주)쇼박스,2,2015-11-19,130,3,우민호,1161602.50,2,387,3,7072501
2,은밀하게 위대하게,(주)쇼박스,10,2013-06-05,123,1,장철수,220775.25,4,343,4,6959083
3,나는 공무원이다,(주)NEW,11,2012-07-12,101,2,구자홍,23894.00,2,20,6,217866
4,불량남녀,쇼박스(주)미디어플렉스,11,2010-11-04,108,1,신근호,1.00,1,251,2,483387


In [42]:
CSV_COLUMN_NAMES = ["time", "genre", "screening_rat", "dir_prev_bfnum", "dir_prev_num", "num_staff", "num_actor", "box_off_num"]
CSV_DEFAULTS = [[0.0], ['na'], ["na"], [0.0], [0.0], [0.0], [0.0], [0.0]]
train_1 = train[CSV_COLUMN_NAMES]


def read_dataset(csv_path):
    def _parse_row(row):
        # decode_csv : csv의 row를 tensor로 변경해주는 api
        fields = tf.decode_csv(records=row, record_defaults=CSV_DEFAULTS)
        
        # dict로 패키징하기. {col_name_1: values_1, col_name_2: values_2}
        features = dict(zip(CSV_COLUMN_NAMES, fields))
        
        # label 분리
        label = features.pop("box_off_num")
        
        return features, label
    
    dataset = tf.data.Dataset.list_files(file_pattern=csv_path)
    dataset = dataset.flat_map(map_func = lambda filename: tf.data.textLineDataset(filenames=filename).skip(count=1))
    
    # (features, label) 포맷으로 파싱
    dataset = dataset.map(map_func = _parse_row)
    
    return dataset


def train_input_fn(csv_path, batch_size=128):
    # csv 파일을 dataset 형식으로
    dataset = read_dataset(csv_path)
    
    # shuffle, repeat(count=None) count가 None이면 끝까지 반복하라는 의미, batch
    dataset = dataset.shuffle(buffer_size=1000).repeat(count=None).batch(batch_size=batch_size)
    
    return dataset

def eval_input_fn(csv_path, batch_size=128):
    dataset = read_dataset(csv_path)
    
    # eval은 shuffle을 하지 않는다. 
    dataset = dataset.batch(batch_size=batch_size)
    
    return dataset

In [38]:
max(train["genre"].unique())
max(train["screening_rat"].unique())

11

In [39]:
# bucket

fc_genre = tf.feature_column.categorical_column_with_identity(key="genre", num_buckets = 12)
fc_screen_rat = tf.feature_column.categorical_column_with_identity(key="screening_rat", num_buckets = 4)




In [ ]:
def get_wide_deep():
    wide_columns = [
        # Feature crosses
        
        # Sparse columns
        fc_genre, fc_screen_rat
    ]

In [27]:
video_id = tf.feature_column.categorical_column_with_identity(
    key='video_id', num_buckets=1000000, default_value=0)
columns = [tf.feature_column.embedding_column(video_id, 9)]
features = {'video_id': tf.sparse.from_dense([[2, 85, 0, 0, 0],
[33,78, 2, 73, 1]])}
print(features["video_id"])
input_layer = tf.keras.layers.DenseFeatures(columns)
dense_tensor = input_layer(features)

SparseTensor(indices=tf.Tensor(
[[0 0]
 [0 1]
 [1 0]
 [1 1]
 [1 2]
 [1 3]
 [1 4]], shape=(7, 2), dtype=int64), values=tf.Tensor([ 2 85 33 78  2 73  1], shape=(7,), dtype=int32), dense_shape=tf.Tensor([2 5], shape=(2,), dtype=int64))


tensorflow.python.data.ops.dataset_ops.RangeDataset